# Analyzing iMessage Conversations

## Notebook Sections: 
---
1. Load
2. Investigate
3. Extract
4. Transform
5. Visualize

In [683]:
import sqlite3
import pandas as pd
import numpy as np
import string
import re

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

## Load chat data from sqlite
----------------------------

In [684]:
# helper functions for connecting to SQLite3 with python
def connect(sqlite_file):
    """ Make connection to an SQLite database file """
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    return conn, c

def close(conn):
    """ Commit changes and close connection to the database """
    conn.close()
    
def table_col_info(cursor, table_name, print_out=False):
    """ Returns a list of tuples with column informations:
    (id, name, type, notnull, default_value, primary_key)"""
    cursor.execute('PRAGMA TABLE_INFO({})'.format(table_name))
    info = cursor.fetchall()

    if print_out:
        print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
        for col in info:
            print(col)

def get_col_names(sql_table):
    '''return a list of column names from sql query table'''
    return list(map(lambda x: x[0], table.description))

## Investigate Database
----------

In [685]:
# sqlite path to chat.db
sqlite_path = '/Users/adamstueckrath/Library/Messages/chat.db'

# connect to db and get cursor to execute sql commands
sqlite_db, sql_command = connect(sqlite_path)

In [686]:
# list of available tables in database, chat.db
table_name_query = '''
                   SELECT name 
                   FROM sqlite_master 
                   WHERE type='table';
                   '''
table_names = sql_command.execute(table_name_query)
table_list = [table[0] for table in table_names]
print(table_list)

['_SqliteDatabaseProperties', 'deleted_messages', 'sqlite_sequence', 'chat_handle_join', 'chat_message_join', 'message_attachment_join', 'handle', 'message', 'chat', 'attachment', 'sync_deleted_messages', 'message_processing_task', 'sync_deleted_chats', 'sync_deleted_attachments', 'kvtable', 'sqlite_stat1']


The contents of these tables should be self-explanatory.

* `attachment` keeps track of any attachments (files, images, audio clips) sent or received, paths to where they are stored, and file format.
* `handle` keeps track of all known recipients (people with whom you previously exchanged iMessages).
* `chat` keeps track of your conversation threads.
* `message` keeps track of all messages along with their text contents, date, and the ID of the recipient.
* `chat_handle_join`, `chat_message_join`, and `message_attachment_join` are all used for joining tables.

While investigating the tables, I found a field in the `message` table that I wanted to explore. 

In [687]:
# print message table schema
message_table = table_col_info(sql_command, 'chat', print_out=True)
print(message_table)


Column Info:
ID, Name, Type, NotNull, DefaultVal, PrimaryKey
(0, 'ROWID', 'INTEGER', 0, None, 1)
(1, 'guid', 'TEXT', 1, None, 0)
(2, 'style', 'INTEGER', 0, None, 0)
(3, 'state', 'INTEGER', 0, None, 0)
(4, 'account_id', 'TEXT', 0, None, 0)
(5, 'properties', 'BLOB', 0, None, 0)
(6, 'chat_identifier', 'TEXT', 0, None, 0)
(7, 'service_name', 'TEXT', 0, None, 0)
(8, 'room_name', 'TEXT', 0, None, 0)
(9, 'account_login', 'TEXT', 0, None, 0)
(10, 'is_archived', 'INTEGER', 0, '0', 0)
(11, 'last_addressed_handle', 'TEXT', 0, None, 0)
(12, 'display_name', 'TEXT', 0, None, 0)
(13, 'group_id', 'TEXT', 0, None, 0)
(14, 'is_filtered', 'INTEGER', 0, None, 0)
(15, 'successful_query', 'INTEGER', 0, None, 0)
(16, 'engram_id', 'TEXT', 0, None, 0)
(17, 'server_change_token', 'TEXT', 0, None, 0)
(18, 'ck_sync_state', 'INTEGER', 0, '0', 0)
(19, 'last_read_message_timestamp', 'INTEGER', 0, '0', 0)
(20, 'ck_record_system_property_blob', 'BLOB', 0, None, 0)
(21, 'original_group_id', 'TEXT', 0, 'NULL', 0)
(22, 

### The field that caught my eye is `is_read`. 

How this field works:
> Apple's iMessage app provides message status updates that let you know when a message has been delivered. It also has a handy feature called Read Receipts that lets you know when someone has read the message.

As you might know, you can disable and enable Read Reciepts on the iMessage app. I wonder if you disable them, can someone still see if their message has been read by looking into the database? 

In [688]:
# query to investigate is_read field 
is_read_query = '''
                SELECT ROWID, text, handle_id, is_read,
                datetime(substr(date, 1, 9) + 978307200, 'unixepoch', 'localtime') as f_date
                FROM message T1
                    INNER JOIN chat_message_join T2
                    ON T2.chat_id=3
                    AND T1.ROWID=T2.message_id
                ORDER BY T1.date;
                '''

# execute sql query in sqlite3 
table = sql_command.execute(is_read_query)

# table.description is description of columns
column_names = get_col_names(table)

# load sql table into pandas dataframe and pass in the column names
is_read_df = pd.DataFrame(table.fetchall(), columns = column_names)

is_read_df.tail(5)

,ROWID,text,handle_id,is_read,f_date
29399,37180,What’s Albertopolis,3,1,2018-04-22 17:37:43
29400,37182,Can you review something for me?,3,0,2018-04-22 17:49:55
29401,37181,Lol I was testing something,3,0,2018-04-22 17:50:09
29402,37183,Sure bb,3,1,2018-04-22 17:55:15
29403,37184,Do you have wifi?,3,0,2018-04-22 17:56:13


### My assumption is correct!

As you can clearly see, you can still see the Read Receipt information on a conversation. I sent the message **Do you have wifi?** and the is_read field is a 0.

I'm not sure if this is the behavior that I expected, but still very interesting. 

## Extract data
-----------

In [689]:
# query to get all messages and attachments for a specific conversation
message_query = '''
                SELECT ROWID, is_from_me, text, handle_id, cache_has_attachments,
                datetime(substr(date, 1, 9) + 978307200, 'unixepoch', 'localtime') as date
                FROM message T1
                INNER JOIN chat_message_join T2
                    ON T2.chat_id=3
                    AND T1.ROWID=T2.message_id
                ORDER BY T1.date;
                '''
# execute sql query in sqlite3 
message_table = sql_command.execute(message_query)

# get list of message column names 
message_table_columns = get_col_names(message_table)

# load sql table into pandas dataframe and pass in the column names
message_df = pd.DataFrame(message_table.fetchall(), columns = message_table_columns)

attachment_query = '''
                   SELECT T1.ROWID, T2.mime_type
                   FROM message T1
                   INNER JOIN chat_message_join T3
                       ON T1.ROWID=T3.message_id
                   INNER JOIN attachment T2
                   INNER JOIN message_attachment_join T4
                       ON T2.ROWID=T4.attachment_id
                   WHERE T4.message_id=T1.ROWID
                       AND (T3.chat_id=3);
                   '''
# execute sql query in sqlite3 
attachment_table = sql_command.execute(attachment_query)

# get list of attachment column names 
attachment_table_columns = get_col_names(attachment_table)

# load sql table into pandas dataframe and pass in the column names
attachment_df = pd.DataFrame(attachment_table.fetchall(), columns = attachment_table_columns)

# join message dataframe and attachment dataframe on ROWID 
chat_df = pd.merge(message_df, attachment_df, how = 'left', on = 'ROWID')

# close sql connection
close(sqlite_db)

# print out the last 5 rows
chat_df.tail(5)


,ROWID,is_from_me,text,handle_id,cache_has_attachments,date,mime_type
29615,37180,0,What’s Albertopolis,3,0,2018-04-22 17:37:43,NaN
29616,37182,1,Can you review something for me?,3,0,2018-04-22 17:49:55,NaN
29617,37181,1,Lol I was testing something,3,0,2018-04-22 17:50:09,NaN
29618,37183,0,Sure bb,3,0,2018-04-22 17:55:15,NaN
29619,37184,1,Do you have wifi?,3,0,2018-04-22 17:56:13,NaN


## Transform data

In [690]:
# house cleaning - drop duplicate columns from dataframe
chat_df.drop(['ROWID'], axis = 1, inplace = True)

# rename columns 
chat_df.rename(columns={'handle_id':'message_id',
                        'is_from_me':'is_sent', 
                        'text':'message',
                        'date':'message_date',
                        'cache_has_attachments':'has_attachment', 
                        'mime_type':'attachment_type'}, inplace = True)

# show the last 5 rows with new column names
chat_df.tail(5)

,is_sent,message,message_id,has_attachment,message_date,attachment_type
29615,0,What’s Albertopolis,3,0,2018-04-22 17:37:43,NaN
29616,1,Can you review something for me?,3,0,2018-04-22 17:49:55,NaN
29617,1,Lol I was testing something,3,0,2018-04-22 17:50:09,NaN
29618,0,Sure bb,3,0,2018-04-22 17:55:15,NaN
29619,1,Do you have wifi?,3,0,2018-04-22 17:56:13,NaN


In [691]:
# add is_received column
# it's the opposite of is_sent column. exmaple: is_sent = 1, is_received = 0
chat_df['is_received'] = chat_df.apply(lambda row: int(not bool(row.is_sent)), axis = 1)

In [692]:
# transform message_date column into datetime.datetime object 
chat_df['message_date'] = pd.to_datetime(chat_df['message_date'])

In [693]:
def add_name_col(is_sent):
    '''return name depending on is_sent column value'''
    if bool(is_sent):
        return 'Adam'
    return 'Bridget'

chat_df['name'] = chat_df.apply(lambda row: add_name_col(row.is_sent), axis = 1)

chat_df.tail(5)

,is_sent,message,message_id,has_attachment,message_date,attachment_type,is_received,name
29615,0,What’s Albertopolis,3,0,2018-04-22 17:37:43,NaN,1,Bridget
29616,1,Can you review something for me?,3,0,2018-04-22 17:49:55,NaN,0,Adam
29617,1,Lol I was testing something,3,0,2018-04-22 17:50:09,NaN,0,Adam
29618,0,Sure bb,3,0,2018-04-22 17:55:15,NaN,1,Bridget
29619,1,Do you have wifi?,3,0,2018-04-22 17:56:13,NaN,0,Adam


## Visualize data

In [694]:
# pie chart for the number of messages sent between us
def plot_pie(labels, values):
    fig = {
        "data":
        [
          {
              "values": values,
              "labels": labels,
              "name": "Messages Betweeen Us",
              "hoverinfo":"label+percent",
              "textinfo":"percent+value",
              "hole": 0,
              "type": "pie",
              "marker": {"line": 
                         {'color':'#000000',
                          'width':2
                         }
                        }
          }
      ],
        "layout": {
            "title":"Number of Messages",
            "annotations": [
                {
                    "font": {"size": 10},
                    "showarrow": False,
                    "text": ""
                }
            ],
      }
    }
    iplot(fig, config = {'displayModeBar': False, 'showLink': False})

    
messages_labels = ['Adam', 'Bridget']
messages_values = [chat_df['is_sent'].value_counts()[1], 
                   chat_df['is_received'].value_counts()[1]]
plot_pie(messages_labels, messages_values)

In [695]:
exclude = set(string.punctuation)
def remove_punctuation(x):
    '''
    function to remove punctuation from a string
    x: any string
    '''
    try:
        x = x.translate(str.maketrans("", "", string.punctuation))
    except:
        pass
    return x
    
# remove attachment/picture messages, and null message values
word_count_df = chat_df[chat_df['has_attachment'] != 1] 
word_count_df = word_count_df[word_count_df['message'].isnull() != True]
word_count_df = word_count_df['message'].to_frame()

# remove punctuation and backwards apostrophe
word_count_df['message'] = word_count_df['message'].apply(lambda x: remove_punctuation(x)) 
word_count_df['message'] = word_count_df['message'].str.replace(r"’", '') 

# lower case all messages and split words on spaces 
word_count_df['message'] = word_count_df['message'].str.lower().str.split(' ')

# flatten nested list to create list of every word 
message_list = [word for sublist in word_count_df.message.tolist() for word in sublist]

# count most used words
words_count = {}
for word in message_list:
    if word in words_count:
        words_count[word] += 1
    else:
        words_count[word] = 1
        
# create list of tuples and sort words by highiest count
words_count = sorted(words_count.items(), key = lambda x: x[1], reverse = True)

# get counts of every word into a dataframe
word_count_df = pd.DataFrame(amount_words, columns = ['word', 'count'])

# remove blank values from word column (i.e. space or empty messages)
word_count_df = word_count_df[word_count_df['word'] != '']

# bar chart for top 10 words sent between us
def plot_bar(labels, values, xaxis, yaxis):
    trace = go.Bar(
        x = labels,
        y = values,
        marker = dict(
            line = dict(
                color = '#00000)',
                width = 2,
            )
        ),
    )
    data = [trace]
    layout = go.Layout(
        title = 'Top 10 Words',
        xaxis = dict(
            title=xaxis,
            autorange = True,
            showgrid = False,
            zeroline = False,
            showline = True,
            autotick = False,
            ticks = labels,
            showticklabels = True
        ),
        yaxis = dict(
            title=yaxis,
            autorange = True,
            showgrid = True,
            zeroline = False,
            showline = True,
            autotick = True,
            ticks = '',
            showticklabels = True
        )
    )
    fig = go.Figure(data = data, layout = layout)
    iplot(fig, config = {'displayModeBar': False, 'showLink': False})
    
# only chart top 10
word_count_df = word_count_df.head(10)
labels = word_count_df['word'].values.tolist()
values = word_count_df['count'].values.tolist()
plot_bar(labels, values, "Word", "Count")

In [696]:
months_map = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
              7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
def get_month(month):
    '''helper function for getting a three letter month for pandas month value'''
    return months_map[month]

# create dataframes for each of us 
adam_df = chat_df[chat_df.is_sent == 1]
bridget_df = chat_df[chat_df.is_received == 1]

# group messages by year and month and counts
adam_df = adam_df['message_date'].groupby([adam_df.message_date.dt.year, adam_df.message_date.dt.month]).agg('count')
bridget_df = bridget_df['message_date'].groupby([bridget_df.message_date.dt.year, bridget_df.message_date.dt.month]).agg('count')

# rename grouped columns and make series in to a dataframe
adam_df = pd.DataFrame({'month':adam_df.index, 'count_a':adam_df.values}) 
bridget_df = pd.DataFrame({'month':bridget_df.index, 'count_b':bridget_df.values}) 

# get three letter month
adam_df['month'] = [get_month(x[1]) for x in adam_df.month.values] 
bridget_df['month'] = [get_month(x[1]) for x in bridget_df.month.values]

# merge with adam dataframe to create a single dataframe
month_count_df = pd.merge(adam_df, bridget_df, how = 'inner', on = 'month')
    
def plot_group_bar(labels, value_a, value_b, title, xaxis, yaxis):
    trace_1 = go.Bar(
        x = labels,
        y = value_a,
        name='Adam',
        marker = dict(
            line = dict(
                color = '#00000)',
                width = 2,
            )
        ),
    )
    
    trace_2 = go.Bar(
        x = labels,
        y = value_b,
        name='Bridget',
        marker = dict(
            line = dict(
                color = '#00000)',
                width = 2,
            )
        ),
    )
    data = [trace_1, trace_2]
    layout = go.Layout(
        title = title,
        barmode = 'group',
        xaxis = dict(
            title=xaxis,
            autorange = True,
            showgrid = False,
            zeroline = False,
            showline = True,
            autotick = False,
            ticks = labels,
            showticklabels = True
        ),
        yaxis = dict(
            title=yaxis,
            autorange = True,
            showgrid = True,
            zeroline = False,
            showline = True,
            autotick = True,
            ticks = '',
            showticklabels = True
        )
    )
    fig = go.Figure(data = data, layout = layout)
    iplot(fig, config = {'displayModeBar': False, 'showLink': False})
    
labels = month_count_df['month'].values.tolist()
a_count = month_count_df['count_a'].values.tolist()
b_count = month_count_df['count_b'].values.tolist()
title = 'Messages by Month'
plot_group_bar(labels, a_count, b_count, title, 'Month', "Messages")

In [697]:
week_map = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 
            4:'Fri', 5:'Sat', 6:'Sun'}
def get_weekday(day):
    '''helper function for getting a three letter weekday for pandas weekday value'''
    return week_map[day]

# create dataframes for each of us 
adam_df = chat_df[chat_df.is_sent == 1]
bridget_df = chat_df[chat_df.is_received == 1]

# group messages by weekday and counts
adam_df = adam_df['message_date'].groupby([adam_df.message_date.dt.weekday]).agg('count')
bridget_df = bridget_df['message_date'].groupby([bridget_df.message_date.dt.weekday]).agg('count') 

# rename grouped columns and make series in to a dataframe
adam_df = pd.DataFrame({'weekday':adam_df.index, 'count_a':adam_df.values}) 
bridget_df = pd.DataFrame({'weekday':bridget_df.index, 'count_b':bridget_df.values})

# get three letter weekday 
adam_df['weekday'] = adam_df.apply(lambda row: get_weekday(row.weekday), axis = 1) 
bridget_df['weekday'] = bridget_df.apply(lambda row: get_weekday(row.weekday), axis = 1)

# merge with adam dataframe to create a single dataframe
week_count_df = pd.merge(adam_df, bridget_df, how = 'inner', on = 'weekday')


# plot group bar chart for messages by weekday 
labels = week_count_df['weekday'].values.tolist()
a_count = week_count_df['count_a'].values.tolist()
b_count = week_count_df['count_b'].values.tolist()
title = 'Messages by Weekday'
plot_group_bar(labels, a_count, b_count, title, "Weekday", "Messages")

In [698]:
# message by hour
# create dataframes for each of us 
adam_df = chat_df[chat_df.is_sent == 1]
bridget_df = chat_df[chat_df.is_received == 1]

# group messages by weekday and counts
adam_df = adam_df['message_date'].groupby([adam_df.message_date.dt.hour]).agg('count')
bridget_df = bridget_df['message_date'].groupby([bridget_df.message_date.dt.hour]).agg('count') 

# rename grouped columns and make series in to a dataframe
adam_df = pd.DataFrame({'hour':adam_df.index, 'count_a':adam_df.values}) 
bridget_df = pd.DataFrame({'hour':bridget_df.index, 'count_b':bridget_df.values})

# merge with adam dataframe to create a single dataframe
hour_count_df = pd.merge(adam_df, bridget_df, how = 'inner', on = 'hour')

# plot group bar chart for messages by hour 
labels = hour_count_df['hour'].values.tolist()
a_count = hour_count_df['count_a'].values.tolist()
b_count = hour_count_df['count_b'].values.tolist()
title = 'Messages by Hour of Day'
plot_group_bar(labels, a_count, b_count, title, "Hour", "Messages")

In [699]:
# Get message over time
adam_df = chat_df[chat_df.is_sent == 1]
bridget_df = chat_df[chat_df.is_received == 1]

# group messages by month, day, year  and counts
adam_df = adam_df['message_date'].groupby([adam_df.message_date.dt.date]).agg('count')
bridget_df = bridget_df['message_date'].groupby([bridget_df.message_date.dt.date]).agg('count')

# rename grouped columns and make series in to a dataframe
adam_df = pd.DataFrame({'date':adam_df.index, 'count_a':adam_df.values}) 
bridget_df = pd.DataFrame({'date':bridget_df.index, 'count_b':bridget_df.values})

# merge dataframes to create a single dataframe
date_count_df = pd.merge(adam_df, bridget_df, how = 'inner', on = 'date')

def plot_scatter(labels, value_a, value_b, title, xaxis, yaxis):
    trace_1 = go.Scatter(
        x = labels,
        y = value_a,
        name='Adam',
        fill = 'tonexty',
        mode= 'lines'
    )
    
    trace_2 = go.Scatter(
        x = labels,
        y = value_b,
        name='Bridget',
        fill = 'none',
        mode= 'lines'
    )
    data = [trace_1, trace_2]
    layout = go.Layout(
        title = title,
        xaxis = dict(
            title = xaxis,
            autorange = True,
            showgrid = True,
            zeroline = True,
            showline = False,
            autotick = True,
            ticks = '',
            showticklabels = True
        ),
        yaxis = dict(
            title = yaxis,
            autorange = True,
            showgrid = True,
            zeroline = True,
            showline = True,
            autotick = True,
            ticks = '',
            showticklabels = True
        )
    )
    fig = go.Figure(data = data, layout = layout)
    iplot(fig, config = {'displayModeBar': False, 'showLink': False})

# plot scatter chart for messages over time 
labels = date_count_df['date'].values.tolist()
a_count = date_count_df['count_a'].values.tolist()
b_count = date_count_df['count_b'].values.tolist()
title = 'Messages by Over Time'
plot_scatter(labels, a_count, b_count, title, "Date", "Messages")

In [700]:
# HEAT MAP !
def plot_heat(labels, values, title, xaxis, yaxis):
    data = [
        go.Heatmap(
            z = values,
            x = labels,
            y = ['Adam','Bridget'],
            colorscale =' Viridis',
            ygap = 0.5,
        )
    ]
    
    layout = go.Layout(
        title = title,
        xaxis = dict(
            title = xaxis,
            autorange = True,
            showgrid = True,
            zeroline = True,
            showline = False,
            autotick = True,
            nticks = 18,
            showticklabels = True
        ),
        yaxis = dict(
            title = yaxis,
            autorange = True,
            showgrid = True,
            zeroline = True,
            showline = True,
            autotick = True,
            ticks = '',
            showticklabels = True
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, config = {'displayModeBar': False, 'showLink': False})
    
# plot scatter chart for messages over time 
labels = date_count_df['date'].values.tolist()
a_count = date_count_df['count_a'].values.tolist()
b_count = date_count_df['count_b'].values.tolist()
title = 'Messages by Over Time'
plot_heat(labels, [a_count, b_count], title, "Date", "Person")